In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import math
import plotly 
from collections import Counter, OrderedDict
import plotly.plotly as py
import plotly.graph_objs as go


In [2]:
def check_missing_values(row):
    counter = 0
    for element in row:
        if element == True:
            counter+=1
    return ("The amoung of missing records is: ", counter)

def plot_heat_map(plot_x, plot_y, plot_z):
    trace = {
        "x": plot_x,
        "y": plot_y,
        "z": plot_z,
        "colorscale": [[0.0, "rgb(158,1,66)"], [0.1, "rgb(213,62,79)"], [0.2, "rgb(244,109,67)"], [0.3, "rgb(253,174,97)"], [0.4, "rgb(254,224,139)"], [0.5, "rgb(255,255,191)"], [0.6, "rgb(230,245,152)"], [0.7, "rgb(171,221,164)"], [0.8, "rgb(102,194,165)"], [0.9, "rgb(50,136,189)"], [1.0, "rgb(94,79,162)"]],
        "type": "heatmap"
    }
    data = go.Data([trace])
    layout = {
        "legend": {
            "bgcolor": "#F5F6F9",
            "font": {"color": "#4D5663"}
        },
        "paper_bgcolor": "#F5F6F9",
        "plot_bgcolor": "#F5F6F9",
        "xaxis1": {
            "gridcolor": "#E1E5ED",
            "tickfont": {"color": "#4D5663"},
            "title": "",
            "titlefont": {"color": "#4D5663"},
            "zerolinecolor": "#E1E5ED"
        },
        "yaxis1": {
            "gridcolor": "#E1E5ED",
            "tickfont": {"color": "#4D5663"},
            "title": "",
            "titlefont": {"color": "#4D5663"},
            "zeroline": False,
            "zerolinecolor": "#E1E5ED"
        }
    }
    fig = go.Figure(data = data, layout=layout)
    return fig

def get_trace(X_pca, data, category, color):
    trace = go.Scatter3d(
        x=X_pca[data.apply(lambda x: True if x==category else False), 0],
        y=X_pca[data.apply(lambda x: True if x==category else False),1],
        z=X_pca[data.apply(lambda x: True if x==category else False),2],
        mode='markers',
        marker=dict(
            size=4,
            line=dict(
                color=color,
                width=0.2
            ),
            opacity=0.8
        ),
        text=data[data.apply(lambda x: True if x==category else False).tolist()]
    )
    return trace

def plot_word_frequency(word_list, plot_title):
    trace1 = {
        "x": list(word_list[0]),
        "y": list(word_list[1]),
        "type": "bar",
    }
    data = go.Data([trace1])
    layout = {
        "title": plot_title,
        "yaxis": {"title": "Frequency"}
    }

    fig = go.Figure(data = data, layout=layout)
    return fig

def get_tokens_and_frequency(token_list):
    counter = Counter(token_list)
    counter = OrderedDict(counter.most_common()) # sort by value
    tokens = counter.keys()
    tokens_count = counter.values()
    return tokens, tokens_count

# 1. Data Preparation

In [3]:
# categories
categories = ['source','sentence', 'score' ,'segment','POS']
f0 = open('amazon_cells_labelled.txt','r')
f1 = open('imdb_labelled.txt','r',encoding = 'utf8')
f2 = open('yelp_labelled.txt','r')

#check data
lines0 = f0.readlines()
print("amazon :\n",[i.split('\t') for i in lines0[:10]])

lines1 = f1.readlines()
print("\nimdb :\n",[i.split('\t') for i in lines1[:10]])

lines2 = f2.readlines()
print("\nyelp :\n",[i.split('\t') for i in lines2[:10]])


amazon :
 [['So there is no way for me to plug it in here in the US unless I go by a converter.', '0\n'], ['Good case, Excellent value.', '1\n'], ['Great for the jawbone.', '1\n'], ['Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!!', '0\n'], ['The mic is great.', '1\n'], ['I have to jiggle the plug to get it to line up right to get decent volume.', '0\n'], ['If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one.', '0\n'], ['If you are Razr owner...you must have this!', '1\n'], ['Needless to say, I wasted my money.', '0\n'], ['What a waste of money and time!.', '0\n']]

imdb :
 [['A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ', '0\n'], ['Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ', '0\n'], ['Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridicu

In [4]:
#check all data
print("amazon dataset: ",len(lines0))
print("imdb dataset: ",len(lines1))
print("yelp dataset: ",len(lines2))
print("\nall dataset: ",len(lines0)+len(lines1)+len(lines2))

amazon dataset:  1000
imdb dataset:  1000
yelp dataset:  1000

all dataset:  3000


Converting Dictionary into Pandas dataframe

In [5]:
categories = ['source','sentence', 'score']
sen = [] 
sco = []
sou = []

In [6]:
for line in lines0:
    sen.append(line[:len(line)-3])
    sco.append(line[len(line)-2:len(line)-1])
    sou.append("amazon")
    
for line in lines1:
    sen.append(line[:len(line)-3])
    sco.append(line[len(line)-2:len(line)-1])
    sou.append("imdb")
    
for line in lines2:
    sen.append(line[:len(line)-3])
    sco.append(line[len(line)-2:len(line)-1])
    sou.append("yelp")

In [7]:
#while True:
#    try:
#        Stop_words
#        stop_words = set(stopwords.words('english'))
#        stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '...', "'s",'/','-','&','``','--',"''",""])
#        for sentence in str:
#             word_segment.append(list(set(word_tokenize(sentence.lower()))-set(stop_words)))
#             pos_token.append(nltk.pos_tag(set(word_tokenize(sentence.lower()))-set(stop_words),lang='eng'))
#        for pos_ in pos_token:
#            pos.append(list([pos for word,pos in pos_]))
#        break;
#    except: 
#        nltk.download()
#        continue

# 2. Converting Dictionary into Pandas dataframe

In [8]:
data_dict = {
             "source": sou,
             "sentence": sen,
             "score": sco,
            }
all_data = pd.DataFrame(data_dict,columns=['source','sentence','score'])

In [9]:
len(all_data)

3000

### 2.1 Familiarizing Data

In [10]:
#first 5 data
all_data[:5]

,source,sentence,score
0,amazon,So there is no way for me to plug it in here i...,0
1,amazon,"Good case, Excellent value.",1
2,amazon,Great for the jawbone.,1
3,amazon,Tied to charger for conversations lasting more...,0
4,amazon,The mic is great.,1


In [11]:
#last 5 data
all_data[-5:]

,source,sentence,score
2995,yelp,I think food should have flavor and texture an...,0
2996,yelp,Appetite instantly gone.,0
2997,yelp,Overall I was not impressed and would not go b...,0
2998,yelp,"The whole experience was underwhelming, and I ...",0
2999,yelp,"Then, as if I hadn't wasted enough of my life ...",0


In [12]:
#search imdb data
all_data[all_data['source'] =="imdb"][:5]

,source,sentence,score
1000,imdb,"A very, very, very slow-moving, aimless movie ...",0
1001,imdb,Not sure who was more lost - the flat characte...,0
1002,imdb,Attempting artiness with black & white and cle...,0
1003,imdb,Very little music or anything to speak of.,0
1004,imdb,The best scene in the movie was when Gerardo i...,1


In [13]:
all_data[0:10]

,source,sentence,score
0,amazon,So there is no way for me to plug it in here i...,0
1,amazon,"Good case, Excellent value.",1
2,amazon,Great for the jawbone.,1
3,amazon,Tied to charger for conversations lasting more...,0
4,amazon,The mic is great.,1
5,amazon,I have to jiggle the plug to get it to line up...,0
6,amazon,If you have several dozen or several hundred c...,0
7,amazon,If you are Razr owner...you must have this!,1
8,amazon,"Needless to say, I wasted my money.",0
9,amazon,What a waste of money and time!.,0


In [14]:
#search yelp data
all_data[all_data['source'] =="yelp"][:5]

,source,sentence,score
2000,yelp,Wow... Loved this place.,1
2001,yelp,Crust is not good.,0
2002,yelp,Not tasty and the texture was just nasty.,0
2003,yelp,Stopped by during the late May bank holiday of...,1
2004,yelp,The selection on the menu was great and so wer...,1


In [15]:
#search positive sentence
all_data[(all_data['score'] == "1")][:5]

,source,sentence,score
1,amazon,"Good case, Excellent value.",1
2,amazon,Great for the jawbone.,1
4,amazon,The mic is great.,1
7,amazon,If you are Razr owner...you must have this!,1
10,amazon,And the sound quality is great.,1


# 3. Check missing value

In [16]:
all_data.isnull().apply(lambda x: check_missing_values(x))

source      (The amoung of missing records is: , 0)
sentence    (The amoung of missing records is: , 0)
score       (The amoung of missing records is: , 0)
dtype: object

### 3.1 Insert missing values

In [17]:
fack_series= pd.Series(pd.np.nan, index=categories)
fack_data = all_data.append(fack_series, ignore_index=True)

In [18]:
len(fack_data)

3001

In [19]:
fack_data.isnull().apply(lambda x: check_missing_values(x))

source      (The amoung of missing records is: , 1)
sentence    (The amoung of missing records is: , 1)
score       (The amoung of missing records is: , 1)
dtype: object

In [20]:
fack_data.dropna(inplace=True)
fack_data.isnull().apply(lambda x: check_missing_values(x))

source      (The amoung of missing records is: , 0)
sentence    (The amoung of missing records is: , 0)
score       (The amoung of missing records is: , 0)
dtype: object

### 3.2 Dealing with Duplicate Data

In [21]:
sum(all_data["sentence"].duplicated())

17

In [22]:
all_data = all_data.drop_duplicates(['sentence'])

In [23]:
len(all_data)

2983

# 4. Data Processing

### 4.1 Sampling

In [24]:
data_sample = all_data.sample(n=1000)
len(data_sample)

1000

In [25]:
data_sample[0:4]

,source,sentence,score
862,amazon,So far it has worked like a charm.,1
2311,yelp,She ordered a toasted English muffin that came...,0
1409,imdb,Vivian Schilling did an excellent job with the...,1
2384,yelp,WILL NEVER EVER GO BACK AND HAVE TOLD MANY PEO...,0


In [26]:
all_category_counts = get_tokens_and_frequency(list(all_data.score))
Sample_category_counts = get_tokens_and_frequency(list(data_sample.score))

In [27]:
py.iplot(plot_word_frequency(all_category_counts, "Score distribution"))

In [29]:
py.iplot(plot_word_frequency(Sample_category_counts, "Score distribution"))


### 4.2 Feature Creation

In [30]:
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', '...', "'s",'/','-','&','``','--',"''",""])

all_data['unigrams'] = all_data['sentence'].apply(lambda x: list(set(word_tokenize(x))-stop_words))

In [31]:
all_data[0:4]['unigrams']

0        [go, way, plug, converter, So, unless, I, US]
1                       [case, Excellent, value, Good]
2                                     [Great, jawbone]
3    [Tied, conversations, lasting, charger, minute...
Name: unigrams, dtype: object

In [32]:
list(all_data[0:3]['unigrams'])

[['go', 'way', 'plug', 'converter', 'So', 'unless', 'I', 'US'],
 ['case', 'Excellent', 'value', 'Good'],
 ['Great', 'jawbone']]

### 4.3 Feature subset selection

In [33]:
count_vect = CountVectorizer(stop_words='english')
X_counts = count_vect.fit_transform(all_data.sentence)

In [34]:
analyze = count_vect.build_analyzer()
analyze(" ".join(list(all_data[0:1].sentence)))

['way', 'plug', 'unless', 'converter']

In [35]:
" ".join(list(all_data[0:1].sentence))

'So there is no way for me to plug it in here in the US unless I go by a converter.'

In [36]:
X_counts.shape

(2983, 4896)

In [37]:
count_vect.get_feature_names()[0:20]

['00',
 '10',
 '100',
 '11',
 '12',
 '13',
 '15',
 '15g',
 '15pm',
 '17',
 '18',
 '18th',
 '1928',
 '1947',
 '1948',
 '1949',
 '1971',
 '1973',
 '1979',
 '1980']

In [38]:
all_data[0:5]

,source,sentence,score,unigrams
0,amazon,So there is no way for me to plug it in here i...,0,"[go, way, plug, converter, So, unless, I, US]"
1,amazon,"Good case, Excellent value.",1,"[case, Excellent, value, Good]"
2,amazon,Great for the jawbone.,1,"[Great, jawbone]"
3,amazon,Tied to charger for conversations lasting more...,0,"[Tied, conversations, lasting, charger, minute..."
4,amazon,The mic is great.,1,"[great, The, mic]"


In [39]:
X_counts[0:5,0:20].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [40]:
count_vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [41]:
count_vect.transform(['00 Something completely new.']).toarray()

array([[1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [42]:
plot_x = ["term_"+i for i in count_vect.get_feature_names()[0:50]]

In [43]:
print(plot_x)

['term_00', 'term_10', 'term_100', 'term_11', 'term_12', 'term_13', 'term_15', 'term_15g', 'term_15pm', 'term_17', 'term_18', 'term_18th', 'term_1928', 'term_1947', 'term_1948', 'term_1949', 'term_1971', 'term_1973', 'term_1979', 'term_1980', 'term_1986', 'term_1995', 'term_1998', 'term_20', 'term_2000', 'term_2005', 'term_2006', 'term_2007', 'term_20th', 'term_2160', 'term_23', 'term_24', 'term_25', 'term_2mp', 'term_30', 'term_30s', 'term_325', 'term_35', 'term_350', 'term_375', 'term_3o', 'term_40', 'term_40min', 'term_42', 'term_44', 'term_45', 'term_4s', 'term_4ths', 'term_50', 'term_5020']


In [44]:
plot_y = ["doc_"+ str(i) for i in (all_data.index)[0:50]]

In [45]:
plot_z = X_counts[0:50, 0:50].toarray()

In [46]:
py.iplot(plot_heat_map(plot_x, plot_y, plot_z))

### 4.4 Dimensionality Reduction

In [47]:
from sklearn.decomposition import PCA

In [48]:
X_reduced = PCA(n_components=3).fit_transform(X_counts.toarray())

In [49]:
X_reduced.shape

(2983, 3)

In [50]:
trace1 = get_trace(X_reduced, all_data["score"], "0", "rgb(71,233,163)")
trace2 = get_trace(X_reduced, all_data["score"], "1", "rgb(229,65,136)")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning:

in the future, boolean array-likes will be handled as a boolean array index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: FutureWarning:

in the future, boolean array-likes will be handled as a boolean array index

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: FutureWarning:

in the future, boolean array-likes will be handled as a boolean array index



In [51]:
data = [trace1, trace2]

In [52]:
layout = go.Layout(
    margin=dict(
        l=0,
        r=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')


### 4.5 Atrribute Transformation / Aggregation

In [53]:
term_frequencies = []
for j in range(0,X_counts.shape[1]):
    term_frequencies.append(sum(X_counts[:,j].toarray()))

In [54]:
term_frequencies[0]

array([1], dtype=int64)

In [55]:
py.iplot(plot_word_frequency([count_vect.get_feature_names(), term_frequencies], "Term Frequency Distribution"))

In [56]:
term_frequencies_log = [math.log(i) for i in term_frequencies]

In [57]:
py.iplot(plot_word_frequency([count_vect.get_feature_names(), term_frequencies_log], "Term Frequency Distribution"))

### 4.6 Discretization and Binarization

In [58]:
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy
mlb = preprocessing.LabelBinarizer()
mlb.fit(all_data.score)


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [59]:
mlb.classes_
all_data['bin_category'] = mlb.transform(all_data['score']).tolist()

In [60]:
all_data[0:9]

,source,sentence,score,unigrams,bin_category
0,amazon,So there is no way for me to plug it in here i...,0,"[go, way, plug, converter, So, unless, I, US]",[0]
1,amazon,"Good case, Excellent value.",1,"[case, Excellent, value, Good]",[1]
2,amazon,Great for the jawbone.,1,"[Great, jawbone]",[1]
3,amazon,Tied to charger for conversations lasting more...,0,"[Tied, conversations, lasting, charger, minute...",[0]
4,amazon,The mic is great.,1,"[great, The, mic]",[1]
5,amazon,I have to jiggle the plug to get it to line up...,0,"[jiggle, plug, right, get, line, I, volume, de...",[0]
6,amazon,If you have several dozen or several hundred c...,0,"[one, fun, contacts, hundred, dozen, sending, ...",[0]
7,amazon,If you are Razr owner...you must have this!,1,"[must, If, owner, Razr]",[1]
8,amazon,"Needless to say, I wasted my money.",0,"[money, say, wasted, Needless, I]",[0]
